# TODO
1. make x_ref, dx_ref optional (or not)
2. impedance control need new forward and inverse, will mess up the original one
3. make sure the two robot don't hit each other
4. jacobian[3:, 3:] or [3:, :3]
5. robot id 0 and 1

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [1]:
# we import useful libraries
import time
from robot_properties_nyu_finger.wrapper import NYUFingerDoubleRobot  # not working anymore
from bullet_utils.env import BulletEnv
import numpy as np
import pybullet
import pinocchio
import pinocchio as pin
import matplotlib
import matplotlib.pyplot as plt

NameError: name 'motor_inertia' is not defined

In [ ]:
# ! Create a Pybullet simulation environment before any robots !
env = BulletEnv()

# Create a robot instance. This adds the robot to the simulator as well.
robot = NYUFingerDoubleRobot()

# Add the robot to the env to update the internal structure of the robot
# at every simulation steps.
env.add_robot(robot)

## Robot Kinematics

In [ ]:
class RobotKinematic:
    def __init__(self, model, data):
        self.model = model
        self.data = data
        self.ee0_id = self.model.getFrameId("finger0_lower_to_tip_joint")
        self.ee1_id = self.model.getFrameId("finger1_lower_to_tip_joint")
    
    def update(self, q, dq):
        '''
        update position and Jacobian based on q, dq
        '''
        self.ee0_pose, self.ee1_pose = self.forward_k(q)
   
        self.body_jacobian_ee0 = pin.computeFrameJacobian(self.model, self.data, q, self.ee0_id)
        self.body_jacobian_ee1 = pin.computeFrameJacobian(self.model, self.data, q, self.ee1_id)

        R_ee0 = self.ee0_pose.rotation
        R_ee1 = self.ee1_pose.rotation

        Ad0 = np.zeros((6, 6))
        Ad0[:3, :3] = R_ee0
        Ad0[3:, 3:] = R_ee0

        Ad1 = np.zeros((6, 6))
        Ad1[:3, :3] = R_ee1
        Ad1[3:, 3:] = R_ee1

        self.oriented_jacobian_ee0 = (Ad0 @ self.body_jacobian_ee0)[:, :3]
        self.oriented_jacobian_ee1 = (Ad1 @ self.body_jacobian_ee1)[:, 3:]

#         self.Jov0 = self.oriented_jacobian_ee0[:3, :3]
#         self.Jov1 = self.oriented_jacobian_ee1[:3, 3:]
        
    def forward_k(self, q):
        # calculate forward kinematics
        pin.forwardKinematics(self.model, self.data, q)
        
        x0 = pin.updateFramePlacement(self.model, self.data, self.ee0_id)
        x1 = pin.updateFramePlacement(self.model, self.data, self.ee1_id)
        return x0, x1
    
    def inverse_k(self, x, dx):
        # todo
        q = x
        dq = dx
        return q, dq

## PD control formula

$\tau = P (\theta_{des} - \theta) + D (\dot{\theta}_{des} - \dot{\theta})$

In [ ]:
class PD_controller:
    def __init__(self, P, D):
        self.update_control(P, D)
    
    def cal_torque(self, q_ref, q_meas, dq_ref, dq_meas):
        '''
        calculate the joint torques
        '''
        error = q_ref - q_meas # the position error for all the joints (it's a 3D vector)
        d_error = dq_ref - dq_meas # the velocity error for all the joints

        # we compute the desired torques as a PD controller
        joint_torques = self.P * error + self.D * d_error

        return joint_torques

    def update_control(self, P, D):
        self.P = P
        self.D = D

## Velocity control formula
$v_O = \dot{x}_{ref} + P (x_{ref} - x_{measured})$\
$\dot{\boldsymbol{\theta}}_{des} = J_{Ov}^{-1} v_O$\
$\tau = D(\dot{\boldsymbol{\theta}}_{des} - \dot{\boldsymbol{\theta}}_{measured})$

In [ ]:
class VelocityController:
    def __init__(self, gain, D):
        self.update_control(gain, D)
        
    def cal_torque(self, x_ref, x_meas, dx_ref, q, dq, jacobian):
        # todo: q not used
        '''
        Given reference position and velocity, calculate and return desired torques
        
        x_ref: reference position
        x_meas: measured position
        dx_ref: reference velocity
        
        return: torques for each joint
        '''
        # velocity part of jacobian
        Jov = jacobian[:3]
        
        # calculated inverse Jacobian
        if np.abs(q[2] < 0.05):
            Jov_inv = np.linalg.pinv(Jov + 1e-4*np.eye(3))
        else:
            Jov_inv = np.linalg.pinv(Jov)
        
        # calculate torque
        dx_des = dx_ref + self.gain * (x_ref - x_meas)
        dq_des = Jov_inv @ dx_des        
        d_error = dq_des - dq
        joint_torques = self.D * d_error
        
        return joint_torques
    
    def update_control(self, gain, D):
        self.gain = gain
        self.D = D

## Impedance control formula
$\tau = J_O^T \Big( K (p_{ref}(t) - p_{measured}) + D (\dot{p}_{ref}(t) - \dot{p}_{measured}) \Big)$

In [ ]:
class ImpedanceController:
    def __init__(self, spring_const, damping):
        self.K = np.array(spring_const)
        self.D = np.array(damping)
    
    def cal_torque(self, x_ref, x_meas, dx_ref, q, dq, jacobian):
        # todo: q not used
        '''
        calculate torque
        '''
        Jov = jacobian[:3]
        dx_meas = Jov.dot(dq)
        
        des_force = self.K.dot(x_ref - x_meas) + self.D.dot(dx_ref - dx_meas)
        joint_torques = Jov.T.dot(des_force)

        return joint_torques

## Simulation

In [ ]:
run_time = 5.0
center = [0, 0, 0]
radius = 0.05
plotting = True
'''
draw a circle in the xz plane
center (list): center of the circle
radius (float): radius of the circle
run_time (int): runtime in seconds
'''
# initialize robot
robot.reset_state(np.array([0., 0., 0., 0., 0., 0.]), np.zeros(robot.nv))
# initialize robot kinematic
rk = RobotKinematic(robot.pin_robot.model, robot.pin_robot.data)
rk.update(*robot.get_state())

# initialize velocity controller for robot 0
gain = 3.
D = np.array([0.3, 0.3, 0.3])
v_controller = VelocityController(gain, D)

# initialize PD controller for robot 1
P = np.array([4,4,3])
D = np.array([.5,.4,.2])
pd_controller = PD_controller(P, D)

# initialize impedance controller for robot 1
K = np.diag([50,50,10])
D = np.diag([5,5,0])
imp_controller = ImpedanceController(K, D)

# setup the center of circular movement
center[0] = rk.ee0_pose.translation[0] - 0.05
center[1] = rk.ee0_pose.translation[1]
center[2] = rk.ee0_pose.translation[2] + 0.1

print(rk.ee0_pose.translation)
print(rk.ee1_pose.translation)

# reset position to avoid yanking
robot.reset_state(np.array([0., -0.5, 1, 0., 0., 0.]), np.zeros(robot.nv))

# delay a while before start
time.sleep(0.5)

# 1 ms for each step
dt = 0.001
num_steps = int(run_time/dt)

if plotting:
    # we store information
    # here we create some arrays that we use to store data generated during the control loop
    measured_positions = np.zeros([num_steps,3]) # will store the measured position
    measured_velocities = np.zeros_like(measured_positions) # will store the measure velocities

    reference_foot_positions = np.zeros([num_steps, 3])
    reference_foot_velocities = np.zeros([num_steps, 3])

    simulation_time = np.zeros([num_steps]) # will store the running time

    x_pos = np.zeros([num_steps]) # will store the x position of the foot (as computed by Forw. Kin.)
    y_pos = np.zeros([num_steps]) # will store the y position of the foot (as computed by Forw. Kin.)
    z_pos = np.zeros([num_steps]) # will store the z position of the foot (as computed by Forw. Kin.)

    x_vel = np.zeros([num_steps]) # will store the x velocity of the foot
    y_vel = np.zeros([num_steps]) # will store the y velocity of the foot
    z_vel = np.zeros([num_steps]) # will store the z velocity of the foot


# now we can enter the main control loop (each loop is 1 control cycle)
for i in range(num_steps):
    # we get the position and velocities of the joints and save them
    q, dq = robot.get_state()
    
    rk.update(q, dq)

    #------------------------------------------------------------------------
    # velocity control for robot 0
    
    # calculate desire position
    t = dt * i
    w = np.pi
    x_ref = np.array([center[0] + radius * np.sin(w * t),
                      center[1],
                      center[2] + radius * np.cos(w * t)])
    dx_ref = np.array([radius * w * np.cos(w * t),
                       0.,
                       -radius * w * np.sin(w * t)])
    # calculate torque
    joint_torques0 = v_controller.cal_torque(x_ref, rk.ee0_pose.translation, dx_ref, q[:3], dq[:3], rk.oriented_jacobian_ee0)
    #------------------------------------------------------------------------
    
    joint_torques1 = np.zeros(3)
    
#     #------------------------------------------------------------------------
#     # PD control for robot 1
#     joint_torques1 = pd_controller.cal_torque(np.array([0,0,np.pi/2]), q[3:], np.zeros(3), dq[3:])
#     #------------------------------------------------------------------------

    #------------------------------------------------------------------------
    # impedance control for robot 1
    x_ref2 = np.array([0.051+0.1, 0.059, 0.05+0.1])
    joint_torques1 = imp_controller.cal_torque(x_ref2, rk.ee1_pose.translation, np.zeros(3), q[3:], dq[3:], rk.oriented_jacobian_ee1)
    #------------------------------------------------------------------------
        
    # we send them to the robot and do one simulation step
    robot.send_joint_command(np.append(joint_torques0, joint_torques1))
    env.step(sleep=True)
    
    if plotting:
        measured_positions[i,:] = q[:3]
        measured_velocities[i,:] = dq[:3]
        simulation_time[i] = dt * i
        x_pos[i], y_pos[i], z_pos[i] = rk.ee0_pose.translation
        reference_foot_positions[i] = x_ref
        reference_foot_velocities[i] = dx_ref
        x_vel[i], y_vel[i], z_vel[i] = rk.oriented_jacobian_ee0[:3] @ dq[:3]
        x_vel[i], y_vel[i], z_vel[i] = rk.oriented_jacobian_ee0[:3] @ dq[:3]

## Plotting for robot 0

In [ ]:
l0 = 0.3
l1 = 0.16
l2 = 0.16
l3 = 0.014

def plot_foot_trajectory(x_pos, y_pos, z_pos):
    """
    plots the position of the foot in 2D and the position of the spatial frame {s}
    we assume that the time varying x variable is in x_pos and that the y variable is in y_pos
    """    
    fig = plt.figure(figsize=(10,3.5))
    plt.subplot(1,2,1)
    plt.plot(x_pos,y_pos)
    plt.xlabel('foot x position [m]')
    plt.ylabel('foot y position [m]')
    plt.plot([0],[0],'o',markersize=15,color='r')
    
    plt.subplot(1,2,2)
    plt.plot(x_pos,z_pos)
    plt.xlabel('foot x position [m]')
    plt.ylabel('foot z position [m]')
    plt.plot([0],[0],'o',markersize=15,color='r')
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot3D(x_pos,z_pos,y_pos)
    plt.xlabel('foot x position [m]')
    plt.ylabel('foot z position [m]')
    ax.plot3D([0],[0],[0],'o',markersize=15,color='r')
    
plot_foot_trajectory(x_pos, y_pos, z_pos)

In [ ]:
font = {'family' : 'serif',
        'size'   : 22}

matplotlib.rc('font', **font)
np.set_printoptions(precision=3, linewidth=200, suppress=True)

def plot_foot_positions(t, measured_foot_positions, desired_foot_positions):
    fig, ax = plt.subplots(3, 1, figsize=(12, 8*3))
    labels = ['X', 'Y', 'Z']

    for i in range(3):
        ax[i].plot(simulation_time, measured_foot_positions[:, i], 
                   lw=4, color='orange',
                   label='measured')
        ax[i].plot(simulation_time, desired_foot_positions[:, i], 
                   lw=4, color='blue',
                   label='reference')

        ax[i].grid()
        ax[i].set_xlabel('Time [s]')
        ax[i].set_ylabel(f'Foot position at {labels[i]}-axis [m]' )
        _ = ax[i].legend(loc='upper right', ncol=1, fancybox=True, shadow=True)
        
measured_foot_positions = np.vstack((x_pos, y_pos, z_pos)).T
plot_foot_positions(simulation_time, measured_foot_positions, reference_foot_positions)


In [ ]:
def plot_foot_velocities(t, measured_foot_velocities, desired_foot_velocities):
    fig, ax = plt.subplots(3, 1, figsize=(12, 8*3))
    labels = ['X', 'Y', 'Z']

    for i in range(3):
        ax[i].plot(simulation_time, measured_foot_velocities[:, i], 
                   lw=4, color='orange',
                   label='measured')
        ax[i].plot(simulation_time, desired_foot_velocities[:, i], 
                   lw=4, color='blue',
                   label='reference')

        ax[i].grid()
        ax[i].set_xlabel('Time [s]')
        ax[i].set_ylabel(f'Foot velocity at {labels[i]}-axis [m/s]' )
        _ = ax[i].legend(loc='upper right', ncol=1, fancybox=True, shadow=True)

measured_foot_velocities = np.vstack((x_vel, y_vel, z_vel)).T
plot_foot_velocities(simulation_time, measured_foot_velocities, reference_foot_velocities)

In [ ]:
env.close()